In [ ]:
knitr::include_graphics("https://slcladal.github.io/images/uq1.jpg")



# Introduction{-}

This tutorial introduces tables and shows how to tabulate data. The entire R markdown document for this tutorial can be downloaded [here](https://slcladal.github.io/table.Rmd). A truly excellent and highly recommendable resource on processing data in R is @wickham2016r [see also @wickham2019welcome. Alternative but also very useful resources are @stander2017enthusing; and @estrellado2020data.

## Preparation and session set up{-}

This tutorial is based on R. If you have not installed R or are new to it, you will find an introduction to and more information how to use R [here](https://slcladal.github.io/IntroR_workshop.html). For this tutorials, we need to install certain *packages* from an R *library* so that the scripts shown below are executed without errors. Before turning to the code below, please install the packages by running the code below this paragraph. If you have already installed the packages mentioned below, then you can skip ahead ignore this section. To install the necessary packages, simply run the following code - it may take some time (between 1 and 5 minutes to install all of the libraries so you do not need to worry if it takes some time).


In [ ]:
# clean current workspace
rm(list=ls(all=T))
# set options
options(stringsAsFactors = F)         # no automatic data transformation
options("scipen" = 100, "digits" = 4) # suppress math annotation
# install libraries
install.packages(c("xlsx", "tidyverse"))


Once you have installed R-Studio and initiated the session by executing the code shown above, you are good to go.

# Gettign started

Tables are one of the most common and important ways to process, handle, and describe data. This tutorial introduces different types of tables, how to load and save different types of tables, as well as how to modify and process tables and tabulated data.

When working with R, there are different kinds or types of tables that have different characteristics. The most common types of tables in R are:

* matrices
* data frames
* tibbles

Matrices can only contain one type of data and all data points will be converted to the type of scale with the lowest information value. For instance, if at least one variables in a table represents characters (or strings), then all other variables are also converted into characters (although they may be numbers). 

Data frames can contain different types of scales and variables are not converted automatically. In this sense, data frames are more flexible and are preferable over matrices.

Tibbles are the tidyverse equivalent of data frames which offer new functions and possibilities of handling and inspecting the data. . 

# Loading tables into R

There are several different functions that allow us to read tabulated data into R. If the data is stored as a txt file, there are various functions to read in the data. The most common functions are `read.delim` and `read.table`. 


In [ ]:
# load data with read.delim
mytable <- read.delim("https://slcladal.github.io/data/mlrdata.txt", header = TRUE)
# show first 6 lines of table
head(mytable)


In [ ]:
# load data with read.delim
myothertable <- read.table("https://slcladal.github.io/data/mlrdata.txt", header = TRUE)
# show first 6 lines of table
head(myothertable)


When a tables should be loaded from Excel, it is necessary to load the `xlsx` package. The example below only works if you exchange the path with your own path to an xls or xlsx file. This is because if we want to load data from the LADAL server, loading the table becomes slightly more complex. You can try it out loading an xls or xlsx file with some xls or xlsx file on your own computer though.



In [ ]:
# load library
library(xlsx)
# load table into R
# WARNING! set your own path!
exceldata <- read.xlsx("https://slcladal.github.io/data/testdata1.xlsx", 1)
# show first 6 lines of table
head(exceldata)


The following section shows how to access and manipulate tables. 

# Dealing with tables in the Tidyverse

The tidyverse is a specific way of writing R code that builds on a family of libraries designed for efficient data science work flows which were developed initially by Hadley Wickham. This new way of writing R code buidls on a shared and underlying design philosophy and grammar. Due to its popularity and ease of use, the tidyverse way to write R code is becoming increasingly popular and we will use it in the following to handle and manipulate tabulated data. 

If you have already loaded data into R and now want to process the data, you typically have to modify the data in some form or another to get the information or format you need. The tidyverse offers very user-friendly, intuitive, and handy functions for processing the data to match the needs of your analysis. To have access to the tidyverse functions for data processing, we load the `tidyverse` package and load and inspect another set of data using the `read.delim` function. The new data is stored as a txt file and has 100 observations (rows) and 3 variables (status, attraction, and money). The data set represents how much money people have spend in someone they were interested in or not (attraction: Interested versus NotInterested) and their own relationship status (status: Single versus Relationship).


In [ ]:
# load libraries
library(tidyverse)
# load new data
newdata <- read.delim("https://slcladal.github.io/data/mlrdata.txt", sep = "\t", header = T)
# inspect data before processing
nrow(newdata); str(newdata); table(newdata$attraction)


The output shows that the table represents 3 variables (status, attraction, and money) and 100 rows which contain 50 people who are interested and 50 who are not interested. We will now check out different ways and functions to process data. 

## Piping

Piping, done with the sequence `%>%`, is a very easy, intuitive, quick, and handy way to process data. Essentially piping means that we take an element that is to the left of the piping symbol and then do something to it; that way, the piping symbol can be translated as *and then*. 

We could, for example, load data and then capitalize the column names and then group the data by status and attraction and then get the mean of money spend on deleting all observations except for the first one. A more formal way to write this would be:

> load %>% capitalize %>% group %>% summarize.

In R this command would look like this:


In [ ]:
# example of a data processing pipeline
pipeddata <- read.delim("https://slcladal.github.io/data/mlrdata.txt", sep = "\t", header = T) %>%
  dplyr::rename(Status = status, Attraction = attraction, Money = money) %>%
  dplyr::group_by(Status, Attraction) %>%
  dplyr::summarise(Mean = mean(Money))
# inspect summarized data
pipeddata


The pipe has worked and we get the resulting summary which shows the mean of the money spend based on  Attraction and Status.

## Selecting and filtering

Among the most frequent procedures in data processing is selecting certain columns or extracting rows based on variable levels. In the tidyverse, this is done by using the `select` and `filter` functions. While `select` allows to extract columns, `filter` is used to extract rows, e.g. to get only observations that have a certain feature. Have a look at the example below.


In [ ]:
# select and filter
reduceddata <- newdata %>%
  # select the columns attraction and money
  dplyr::select(attraction, money) %>%
  # extract rows which represent cases where the person was interested in someone
  dplyr::filter(attraction == "Interested")
# inspect new table
nrow(reduceddata); table(reduceddata$attraction)


We have now reduced the data by excluding status (we have only selected attraction and money) and we have removed those 50 data rows of people who were not interested. The `select` function (like most other tidyverse functions) can also be used together with a minus sign which causes a column to be removed, thus `dplyr::select(-money)` would remove the money column (see below).



In [ ]:
# select and filter
datawithoutmoney <- newdata %>%
  # remove money
  dplyr::select(-money) 
# inspect data
head(datawithoutmoney)


Selecting and filtering are extremely powerful functions that can also be combined with other functions. But before we discuss more complex issues, we will have a look at how we can change columns.

## Changing data and adding columns

Changing and adding data is done with the `mutate` function. The `mutate` functions requires that we specify a column name - if we use the same name as the column we are changing, then we change the column but if we specify another column mame, then a new column is created. 

We will now create a new column (Spendalot) in which we encode if the person has spend a lot of money (100 AUD or more) on the present or not (less than 100 AUD).


In [ ]:
# creating a new column
newdata <- newdata %>%
  dplyr::mutate(Spendalot = ifelse(money >= 100, "Alot", "Alittle")) 
# inspect data
head(newdata)


The table now has a new column (Spendalot) because we have specified a column name that did not exist yet - had we written `dplyr::mutate(money = ifelse(money >= 100, "Alot", "Alittle"))` then we would have changed the money column and replaced the money values with the labels *Alot* and *Alittle*.

## Renaming columns

Oftentimes, column names are not really meaningful or incoherent which makes it easier to wrap your head around what the values in a column refer to. The easiest way around this is rename columns which is, fortunately very simple in the tidyverse. While the column names of our example table are meaningful, I want to capitalize the first letter of each column name. This can be done as follows.


In [ ]:
# renaming columns
newdata <- newdata  %>%
  dplyr::rename(Status = status, Attraction = attraction, Money = money)
# inspect data
head(newdata)


The renaming was successful as all column names now bein with a capital letter.

## Grouping and summarising

In contrast to mutate, which retains the number of rows, summarizing creates new columns but collapses rows and only provides the summary value (or values if more than one summary is specified). Also, columns that are not  grouping variables are removed. 

Summarizing is particularly useful when we want to get summaries of groups. We will modify the example from above and extract the mean and the standard deviation of the money spend on presents by relationship status and whether the giver was attracted to the givee. 


In [ ]:
#grouping and summarizing data 
datasummary <- newdata %>%
  dplyr::group_by(Status, Attraction) %>%
  dplyr::summarise(Mean = round(mean(Money), 2), SD = round(sd(Money), 1))
# inspect summarized data
datasummary


## Gathering and Spreading

One very common problem is that data - or at least parts of it - have to be transformed from long to wide format or vice versa. In the tidyverse, this is doen using the `gather` and `spread` function. We will convert the summary table shown above into a wide format (we also remove the SD column as it is no longer needed) 


In [ ]:
# converting data to wide format 
widedata <- datasummary %>%
  # remove SD column
  dplyr::select(-SD) %>% 
  # convert into wide format
  tidyr::spread(Attraction, Mean)
# inspect wide data
widedata


We can re-convert the wide into a long format using the `gather` function. 



In [ ]:
# converting data to long format 
longdata <- widedata %>%
  # convert into long format
  tidyr::gather(Attraction, Money, Interested:NotInterested)
# inspect wide data
longdata


There are many more useful functions for processing, handling, and summarizing tables but this should suffice to get you started.



# Citation & Session Info {-}

Schweinberger, Martin. 2020. *Handling tables in R*. Brisbane: The University of Queensland. url: https://slcladal.github.io/table.html (Version 2020.12.03).


In [ ]:
@manual{schweinberger2020tabr,
  author = {Schweinberger, Martin},
  title = {Handling tables in R},
  note = {https://slcladal.github.io/table.html},
  year = {2020},
  organization = "The University of Queensland, School of Languages and Cultures},
  address = {Brisbane},
  edition = {2020/12/03}
}


In [ ]:
sessionInfo()



***

[Go back to the main page](https://slcladal.github.io/index.html)

***

# References {-}
